# Generalized Markov Chains

Let's talk a little bit about how these Markov chains will be formatted.

We want to build chains that make sense in terms of object size and in terms of melodic structure. 

On one extreme, given a single note in the 'current step', it might make sense to store this note's absolute (0-127) value as a key, and to store as its value a dictionary representing a PMF of the possible 'next steps'. 

On the other extreme, if we have three or four notes taken as the 'current step', it might make more sense to 'normalize' the sequence so that it starts at value 0 before storing it as a key. Otherwise, there will likely be very few occurrences of the sequence, leading to quite sparse PMFs as the values and rather large data structures to hold all of the information. Additionally, recentering will help to group together note structures in different musical key signatures that truly represent melodically identical sequences.

As a bit of a compromise, we will store 'current steps' with length one or two as **absolute** and with length greater than two as **relative** (i.e. recentered with the first note in the sequence equal to 0).

Let's start by getting the relevant code from the last Markov Chain notebook.

In [1]:
import pickle

In [2]:
with open('all_melodies.pkl', 'r') as f:
    all_melodies = pickle.load(f)
with open('all_rhythms.pkl', 'r') as g:
    all_rhythms = pickle.load(g)
assert len(all_melodies) == len(all_rhythms)

In [3]:
class Markov(object):
    """A container for holding Markov chains of various orders, 
    where the events in the given 'current state' may occur temporally 
    before and/or after the note to be examined as the 'next state'.
    """
    
    def __init__(self, before = 0, after = 0, mode = 0):
        """Initialize a Markov object with `before` notes before the
        space to be filled, `after` notes after the space to be filled,
        and with `mode` equal to 0, 1 or 2 according to whether the 'current
        state' contains notes only before, only after, or both before and after
        the note to be filled in the 'next state'.
        
        Inputs: 
        before - int - number of notes before next state
        after - int - number of notes after next state
        mode - int in range(3) - mode of the chain(see above)
        
        Outputs - Markov object
        """
        
        if mode == 0:
            assert before != 0 and after == 0
        elif mode == 1:
            assert before == 0 and after != 0
        else:
            assert before != 0 and after != 0

        self.before = before
        self.after = after
        self.mode = mode
        self.state_dict = {}
        
    def add_data(self, seq, result):
        """Add one (current state -> next state) instance to the dictionary.
        Store each instance as a tally, to be normalized later.
        
        Inputs:
        seq - if mode = 0, seq is a tuple of length before
              if mode = 1, seq is a tuple of length after
              if mode = 2, seq is a list of length two, with first element
                           a tuple of length before and with second element
                           a tuple of length after
        result - int - single event
        
        Outputs:
        None
        """
        
        if self.mode == 0:
            assert isinstance(seq, tuple) and len(seq) == self.before
        elif self.mode == 1:
            assert isinstance(seq, tuple) and len(seq) == self.after
        else:
            assert (isinstance(seq, tuple) and len(seq) == 2 and
                    isinstance(seq[0], tuple) and len(seq[0]) == self.before and
                    isinstance(seq[1], tuple) and len(seq[1]) == self.after)
            
        if seq not in self.state_dict:
            self.state_dict[seq] = {result: 1}
        elif result in self.state_dict[seq]:
            self.state_dict[seq][result] += 1
        else:
            self.state_dict[seq][result] = 1
            
    def normalize(self):
        """Convert the state_dict dictionary from counts to probabilities.
        
        Inputs: None
        
        Outputs: None
        """
        
        for seq in self.state_dict:
            sum = 0
            for result in self.state_dict[seq]:
                sum += self.state_dict[seq][result]
            for result in self.state_dict[seq]:
                self.state_dict[seq][result] /= float(sum)

In [4]:
for melody in all_melodies:
    for i in range(len(melody)):
        assert isinstance(melody[i], list)

Note the change here: we will call a helper function to recenter when the full_length variable is 3 or longer.

In [5]:
import itertools

def iterate_melody(mel, mark, before, after):
    """Adds one new observation to the Markov chain training process.
    
    Inputs: 
    mel - a list of lists representing a melody sequence
    mark - a Markov object
    before - integer, how many notes before next state
    after - integer, how many notes after next state
    
    Outputs:
    None (instead adds data to markov object directly)
    """
    
    full_length = before + after + 1
    for i in range(len(mel) - full_length):
        for seq in itertools.product(*mel[i:i + full_length]):
            before_seq, after_seq, val = seq[:before], seq[-after:], seq[before]
            if full_length > 3:
                before_seq, after_seq, val = recenter(before_seq, after_seq, val)
            mode = mark.mode
            if mode == 0:
                mark.add_data(before_seq, val)
            elif mode == 1:
                mark.add_data(after_seq, val)
            else:
                mark.add_data((before_seq, after_seq), val)
                
def recenter(before, after, val):
    """Recenters sequence(s) so that the first note valued encountered is 0 and every other
    note value is relative to the first.
    
    Ex: before = [45, 47], after = [44, 45], val = 40 returns ([0, 2], [-1, 0], -5)
        before = [], after = [100, 100, 99], val = 100 returns ([], [0, 0, -1], 0)
        
    Inputs: before and after are lists of note values with combined length at least 3,
            val is a note value
    
    Outputs: Tuple of length 3 of the scaled before, after, and val note values.
    """
    
    assert len(before) + len(after) > 2
    if len(before) == 0:
        to_subtract = after[0]
    else:
        to_subtract = before[0]
    return (tuple(map(lambda x: x - to_subtract, before)), 
            tuple(map(lambda x: x - to_subtract, after)), 
            val - to_subtract)

Now we can try to loop through and make nine distinct Markov Chains:

In [6]:
for before in range(4):
    for after in range(4):
        if before + after > 4:
            continue
        if before == 0:
            if after == 0:
                continue
            mode = 1
        elif after == 0:
            mode = 0
        else:
            mode = 2
        mark = Markov(before, after, mode)
        for k in range(len(all_melodies)):
            iterate_melody(all_melodies[k], mark, before = before, after = after)
            if k % 1000 == 0:
                print k
        mark.normalize()
        with open('markov' + str(before) + str(after) + str(mode) + '.pkl', 'w') as f:
            pickle.dump(mark, f)
        print before, after

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
0 1
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
0 2
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000

Supremely cool! The full pickled size of these files is about 76 MB.